In [ ]:
import re
from nltk.corpus import stopwords
from scipy import sparse
import numpy as np
import holidays
from utils import ExtractCalendar
from datetime import date
import re
import pandas as pd
from datetime import date

stop_words = stopwords.words('english')
# stop_words.extend(
#         ['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get',
#          'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot',
#          'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come',
#          'aci'])

texts = open('corpus.txt', 'r')
texts = texts.readlines()
texts


# text = ''.join([words for words in text])

def clean_text(
        string: str,
        punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
        stop_words=stopwords.words('english')
) -> str:
    string = re.sub(r'https?://\S+|www\.\S+', '', string)
    string = re.sub(r'<.*?>', '', string)

    for x in string.lower():
        if x in punctuations:
            string = string.replace(x, "")

    string = string.lower()
    string = ' '.join([word for word in string.split() if word not in stop_words])

    string = re.sub(r'\s+', ' ', string).strip()
    string = string.split()
    return string


window = 2
word_lists = []
all_text = []

for text in texts:
    text = clean_text(text)
    all_text += text
    for i, word in enumerate(text):
        for w in range(window):
            if i + 1 + w < len(text):
                word_lists.append([word] + [text[(i + 1 + w)]])
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])
            # print(i,word)
word_lists

baskets


def create_unique_word_dict(text: list) -> dict:
    """

    :param text:
    :return:
    """
    words = list(set(text))

    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({word: i})

    return unique_word_dict


unique_word_dict = create_unique_word_dict(all_text)

# def create_one_hot_vectors(unique_words:dict)
# One-hot-encoding
n_words = len(unique_word_dict)
words = list(unique_word_dict.keys())

X = []
Y = []

for word_pair in word_lists:
    X.append((word_pair[0] == np.array(words)) * 1)
    Y.append((word_pair[1] == np.array(words)) * 1)
X = np.asarray(X)
Y = np.asarray(X)
word_lists
X.shape
Y.shape
from tensorflow import keras
from keras import Input
from keras.models import Model
from keras.layers import Dense

# import tensorflow as tf

embed_size = 2
X.shape[1]
inp = Input(shape=X.shape[1], )
inp_layer = Dense(embed_size, activation='linear', )(inp)
hidden_layer = Dense(units=Y.shape[1], activation='softmax')(inp_layer)

model = Model(inputs=inp, outputs=hidden_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x=X, y=Y, batch_size=256, epochs=1000)

weights = model.get_weights()[0]
embedding_dict = {}
for word in words:
    embedding_dict.update({word: weights[unique_word_dict[word]]})
embedding_dict